In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import filter_ops

import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import numpy as np
import timeit

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import datetime
def ts(fmt='%d-%m-%y-%H:%M:%S'):
    return datetime.datetime.now().strftime(fmt)

def tf_config(devices='0'):
      return tf.ConfigProto(allow_soft_placement=False,
                            gpu_options=tf.GPUOptions(allow_growth=True,
                                                      visible_device_list=devices))
def relative_error(x, y, norm=np.abs, eps=1e-6):
    return norm(x - y) / (np.maximum(norm(x), norm(y)) + eps)

# initialization
outputs = {}
N, C, F = 64*64, 8, 3
unaries_shape = [N, C]
features_shape = [N, F]

np.random.seed(123)
np_compat = np.random.normal(0.0, 1.0, size=(C, C)).astype(np.float32)
np_compat = np_compat.dot(np_compat.T)
np_unaries = np.random.normal(size=unaries_shape).astype(np.float32)
np_features = np.random.normal(0.0, 10.0, size=features_shape).astype(np.float32)

## our tf/cpp implementation

In [2]:
def softmax(logits, axis=None):
    l_exp = tf.exp(logits - tf.reduce_max(logits, axis=axis, keep_dims=True))
    return l_exp / tf.reduce_sum(l_exp, axis=axis, keep_dims=True)

def meanfield_iteration(q, unaries, lattice, compat):
    """a single iteration of MF inference
    Args:
        q - [N, C] current probabilities
        unaries - [N, C]
        lattice - result of `filter_ops.init_lattice`
        compat - [C, C] symmetric (!) matrix
    Returns:
        probabilities at next step
    """
    pairwise = - filter_ops.ph_filter(q, lattice)
    if len(compat.shape) == 2:
        pairwise = tf.matmul(pairwise, compat)
    else:
        pairwise = pairwise * compat
    q_nat = - (unaries + pairwise)
    return softmax(q_nat, axis=-1)

def meanfield_op(unaries, lattice, compat, num_iters):
    q_init = softmax(-unaries, axis=-1)    
    
    def _iter(prev, it):
        return meanfield_iteration(prev, unaries, l)
    
    q = tf.foldl(lambda prev, it: meanfield_iteration(prev, unaries, 
                 tf.range(0, num_iters), 
                 initializer=q_init,
                 parallel_iterations=32)
    return q

In [3]:
tf.reset_default_graph()

num_iters = 10

# negative log-probability
unaries = tf.convert_to_tensor(np_unaries)
features = tf.convert_to_tensor(np_features)
compat = tf.convert_to_tensor(np_compat)
lattice = filter_ops.permuto_init(np_features)

q = meanfield_op(unaries, lattice, compat, num_iters)
tf_out = filter_ops.ph_filter(inputs, lattice)

with tf.Session(config=tf_config()) as sess:
    print(f'[{ts()}] running')
    _q_tf = sess.run(q)
    _lattice, _tf_out = sess.run([lattice, tf_out])
    print(f'[{ts()}] done!')

NameError: name '_iter' is not defined

## testing how close the results are 

In [ ]:
import numpy as np
import pydensecrf.densecrf as dcrf

crf = dcrf.DenseCRF(N, C)
crf.setUnaryEnergy(np_unaries.T.copy())

if isinstance(np_compat, float):
    crf.addPairwiseEnergy(np_features.T.copy(), 
                          np_compat,
                          dcrf.CONST_KERNEL,
                          dcrf.NO_NORMALIZATION)
else:
    crf.addPairwiseEnergy(np_features.T.copy(), 
                          -np_compat,
                          dcrf.CONST_KERNEL,
                          dcrf.NO_NORMALIZATION)
    

print(f'[{ts()}] running')

q, tmp1, tmp2 = crf.startInference()
_q_init_py = np.array(q).T
for i in range(num_iters):
    crf.stepInference(q, tmp1, tmp2)
_q_py = np.array(q).T

print(f'[{ts()}] done!')

print('difference in q:', np.abs(_q_py - _q_tf).max())

print(np.unique(np.argmax(_q_py, axis=1), return_counts=True))
print(np.unique(np.argmax(_q_tf, axis=1), return_counts=True))

## reference numpy implementation

In [ ]:
# first, let's try numpy implementation
def np_softmax(logits, axis=None):
    l_max = np.max(logits, axis=axis, keepdims=True)
    l_exp = np.exp(logits - l_max)
    return l_exp / np.sum(l_exp, axis=axis, keepdims=True)

def np_splat(inputs, offsets, weights, nbs):
    N, C = inputs.shape
    F = weights.shape[1] - 1
    M = nbs.shape[0]
    # splatting
    ## compute inputs multiplied by the weights
    weighted_inputs = np.matmul(weights[:N,:,np.newaxis], 
                                inputs[:N,np.newaxis,:])
    weighted_inputs = weighted_inputs.reshape([-1, C])
    ## sum up at corresponding indices (update with duplicatess)
    idxs = offsets[:N,:F+1].reshape((-1,))+1
    values = np.zeros([M+2, C])
    # TODO: this is the only op that is hard to do with grads
    np.add.at(values, idxs, weighted_inputs)
    return values

def np_blur(inputs, values_in, offsets, weights, nbs):
    F = weights.shape[1] - 1
    values = values_in.copy()
    # NOTE: we actually ignore the last update?
    for j in range(F+1):
        n1 = values[nbs[:,j,0]+1]
        n2 = values[nbs[:,j,1]+1]
        values[1:-1] += 0.5 * (n1 + n2)
    return values

def np_slice(inputs, values, offsets, weights, nbs):
    N, C = inputs.shape
    F = weights.shape[1] - 1
    M = nbs.shape[0]
    alpha = 1.0 / (1.0 + 2.0**(-F))
    idxs = offsets[:N,:F+1].reshape((-1,))+1
    w = weights[:N,:,np.newaxis]
    v = values[idxs].reshape((N, F+1, C))
    return np.sum(alpha * w * v, axis=1)

def np_filter(inputs, offsets, weights, nbs):
    v_splat = np_splat(inputs, offsets, weights, nbs)
    v_blur = np_blur(inputs, v_splat, offsets, weights, nbs)
    return np_slice(inputs, v_blur, offsets, weights, nbs)
    
offsets = _lattice.offsets
weights = _lattice.weights
nbs = _lattice.neighbours
inputs = np_softmax(-np_unaries, axis=1)

np_out = np_filter(inputs, *_lattice)
err = np.max(np.abs(np_out - _tf_out))
print(f'max error tf vs numpy: {err}')